In [47]:
# import libraries
import time

import pandas as pd
import requests
from bs4 import BeautifulSoup
from selenium.webdriver import Chrome
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys


# Open and load

In [201]:
url = "https://www.vrbo.com/search?regionId=235&semcid=VRBO-US.B.GOOGLE.BD-c-EN.VR&semdtl=a118921852681.b1145236417018.g1kwd-522436780534.e1c.m1CjwKCAiA74G9BhAEEiwA8kNfpQI3jfqFv5rIIHkaPzgVrlALK5ybAW4IImA5xRUpnmTkYFzWLtfjJxoCygwQAvD_BwE.r12e213678c473a12ebda4545133f2c07ecb5fc68c81df9a7bed9eff41fcdc4ffc.c1ioSXP-1MdtLmUk2KclVEuw.j19073655.k121160.d1639955023346.h1p.i1.l1.n1.o1.p1.q1.s1.t1.x1.f1.u1.v1.w1&gad_source=1&gclid=CjwKCAiA74G9BhAEEiwA8kNfpQI3jfqFv5rIIHkaPzgVrlALK5ybAW4IImA5xRUpnmTkYFzWLtfjJxoCygwQAvD_BwE&destination=North%20Carolina%2C%20United%20States%20of%20America&isInvalidatedDate=false&theme=&userIntent=&adults=2&children=&latLong=35.719758%2C-79.453125&mapBounds=&pwaDialog=&amenities=&sort=RECOMMENDED"

In [202]:
driver = Chrome()
# Open the url
driver.get(url)

In [203]:
"""
# The first button that appears once you open the site (Will you help us improve our site experience by answering a few questions?)
no_button = driver.find_element(
    By.XPATH,
    "//*[@class = 'QSIWebResponsiveDialog-Layout1-SI_2gkChTmM30eirR4_button QSIWebResponsiveDialog-Layout1-SI_2gkChTmM30eirR4_button-2 QSIWebResponsiveDialog-Layout1-SI_2gkChTmM30eirR4_button-compact QSIWebResponsiveDialog-Layout1-SI_2gkChTmM30eirR4_button-border-radius-completely-rounded']",
)
no_button.click()
"""

'\n# The first button that appears once you open the site (Will you help us improve our site experience by answering a few questions?)\nno_button = driver.find_element(\n    By.XPATH,\n    "//*[@class = \'QSIWebResponsiveDialog-Layout1-SI_2gkChTmM30eirR4_button QSIWebResponsiveDialog-Layout1-SI_2gkChTmM30eirR4_button-2 QSIWebResponsiveDialog-Layout1-SI_2gkChTmM30eirR4_button-compact QSIWebResponsiveDialog-Layout1-SI_2gkChTmM30eirR4_button-border-radius-completely-rounded\']",\n)\nno_button.click()\n'

In [204]:
# Find and click the done button
done_button = driver.find_elements(
    By.XPATH,
    "//*[@class = 'uitk-button uitk-button-medium uitk-button-has-text uitk-button-primary uitk-layout-flex-item']",
)
done_button[0].click()

# Collect URLs

In [205]:
all_links = []  # This will have all the links

# Create the scrollable section
scrollable_section = driver.find_element(
    By.XPATH,
    "//*[@class = 'uitk-layout-grid-item scrollable-result-section uitk-scrollable uitk-scrollable-vertical']",
)

len_final_message = 0

# for loop to itirate over pages
while len_final_message == 0:
    time.sleep(5)  # Till the new page loads
    for index in range(4):  # 4 steps: 25%, 50%, 75%, 100%
        scroll_height = driver.execute_script(
            "return arguments[0].scrollHeight;", scrollable_section
        )
        scroll_position = (index + 1) * (scroll_height / 4)  # Move in quarters

        driver.execute_script(
            "arguments[0].scrollTop = arguments[1];",
            scrollable_section,
            scroll_position,
        )

        time.sleep(3)  # Allow content to load

    links = driver.find_elements(
        By.XPATH, "//*[@class = 'uitk-card-link']"
    )  # Links in the page
    all_links.extend(links)  # ADD the values to the entire links list

    # Next pages button
    next_button = driver.find_elements(
        By.XPATH,
        "//*[@class = 'uitk-button uitk-button-medium uitk-button-only-icon uitk-button-primary uitk-spacing uitk-spacing-margin-inlinestart-four']",
    )
    driver.execute_script(
        "arguments[0].click();", next_button[0]
    )  # Clicks the next button

    # End loop factor is the appearing of final message
    final_message = driver.find_elements(
        By.XPATH,
        '//h3[text()="We don\'t have any more properties that match your search criteria"]',
    )

    len_final_message = len(final_message)

In [206]:
def infinite_scroll():
    """
    Continuously scrolls through a scrollable section and clicks the next button.
    This function uses JavaScript to scroll to the bottom of a specified scrollable section
    and then clicks a button to load more content. It repeats this process indefinitely.
    Note:
        - The function assumes that the `driver` and `scrollable_section` variables are defined
          and accessible within the scope where this function is called.
        - The function also assumes that the `By` class from `selenium.webdriver.common.by` and
          the `time` module are imported.
    Raises:
        IndexError: If the next button is not found in the DOM.
    """
    while True:
        driver.execute_script(
            "arguments[0].scrollTop = arguments[0].scrollHeight;", scrollable_section
        )

        next_button = driver.find_elements(
            By.XPATH,
            "//*[@class = 'uitk-button uitk-button-medium uitk-button-only-icon uitk-button-primary uitk-spacing uitk-spacing-margin-inlinestart-four']",
        )

        driver.execute_script(
            "arguments[0].click();", next_button[0]
        )  # Clicks the next button
        time.sleep(4)